# Generators
## Overview

When the body of a function contains one or more occurrences of the keyword **yield**, the function is named a "**generator**".

When a generator is called, the function body does not execute. Instead, calling the generator returns a special **iterator** object that wraps the function body, the set of its local variables (including its parameters), and the current point of execution, which is initially the start of the function.

When the **next()** method of this iterator object is called, the function body executes up to the next **`yield`** statement, which takes the form:

`yield expression`


When a `yield` statement executes, the function is frozen with its execution state and local variables intact, and the expression following `yield` is returned as the result of the `next()` method.

On the next call to `next()`,execution of the function body resumes where it left off, again up to the next `yield` statement.

If the function body ends or executes a `return` statement, the iterator raises a `StopIteration` to indicate that the iterator is finished.

**Note**: return statements in a generator cannot contain expressions.

The most common way to use an iterator is to loop on it with a `for` statement, you typically call a generator in the same way:

`for avariable in somegenerator(arguments):`

Here is a generator that works somewhat like the built-in `range()` function, but returns a sequence of floating-point values instead of a sequence of integers:

In [1]:
def frange(start, stop, step=1.0):
    while start < stop:
        yield start
        start += step
        
for i in frange (2.3, 8.67):
    print(i, end=" ")

2.3 3.3 4.3 5.3 6.3 7.3 8.3 

## Generator Expressions

Some simple generators can be coded succinctly as expressions using syntax similar to **list comprehensions** but with parentheses instead of square brackets.

Generator Expressions look like list comprehensions, but returns a generator back instead of a list.

These **generator expressions** are designed for situations where the generator is used right away by an enclosing function or `for` loop.

Generator expressions are more compact but less versatile than full generator definitions and tend to be more memory friendly than equivalent list comprehensions.

Instead of:

In [23]:
def power2(nb):
    for i in range(1,nb+1):
        yield i*i
for val in power2(10):
    print(val, end=" ")


1 4 9 16 25 36 49 64 81 100 

you can write:

In [24]:
for val in (i*i for i in range(1,11)):
    print(val, end=" ")

1 4 9 16 25 36 49 64 81 100 

The utility of generator expressions is greatly enhanced when combined with reduction functions like `sum()`, `min()`, `max()` or `itertools.reduce()`.

The following summation code will build a full list of squares in memory, iterate over those values, and, when the reference is no longer needed, delete the list:


In [5]:
sum([x*x for x in range(10)])

285

Memory is conserved by using a generator expression instead:


In [6]:
sum(x*x for x in range(10))

285

The syntax requires that a generator expression always needs to be directly inside a set of parentheses and cannot have a comma on either side.

This means that you can write: 
       `sum((x**2 for x in range(10)))`
   or
       `sum(x**2 for x in range(10))`
       
but you would have to write:

In [8]:
import functools
import operator
print(functools.reduce(operator.add, (x**2 for x in range(10))))

285


## Generator as co-routines

In languages that provide for generators, an important feature is the ability to pass a value back into the generator. This allows for supporting a programming feature called **coroutines**.

In order to make the generators more powerful, the designers of Python have added in Python 2.5 the ability to pass data back into the generator.

We’ve seen that generators allow us to pull data and pause execution from a function context. Coroutines allow us to push data. In this case, the `yield` statement basically means "Wait until you get some input data". 

If, from the perspective of the generator, you think of the `yield` as calling something, then the concept is easy; you just save the results of yield:

`result = yield`

From the perspective of the caller of the generator, this statement returns control back to the caller, just as before. 

From the perspective of the generator, when the execution comes back into the generator, a value will come with it (in this case, the generator saves it into the variable `result`).

Where does the value come from? The caller must call a method called **`send()`** to pass a value back into the generator. The method `send()` behaves just like the method `next()`, except that it passes a value.

Before we can do this, however, we must first ***prime*** the coroutine with a call to either `send(None)` or `__next__()`: a coroutine can't receive a value right away; it must first run through all its code leading up to its first `yield`.

As with a generator, a coroutine is finished when it either reaches the end of its normal execution flow, or when it hits a return statement.

Iterator objects also have a **`close()`** method (new in Python 2.5). This method frees up the resources for the generator and finishes the coroutine.

If you call `next()` or `send()` again after calling `close()`, you'll get a `StopIteration` exception.


In [16]:
def check_name_exists():
    phone = {"Dennis":89766, "Yann":78887, "Muriel":87765, "Tony":54443, "Julia":76665}
    print("Ready to check for names.")
    while True:
        name = yield
        if name in phone:
            print(f"The phone of {name} is {phone.get(name)}")
        else:
            print(f"Sorry, {name} not found")
            
coro = check_name_exists() 
next(coro)

coro.send("Marco")
coro.send("Yann")
coro.close() # To stop the coroutine
coro.send("Dennis") # Exception raised !

Ready to check for names.
Sorry, Marco not found
Sorry, None not found
The phone of Yann is 78887


StopIteration: 

### Throwing exceptions to a generator

Generators and coroutines also have a **`throw()`** method, which is used to raise an exception at the place they're paused.


In [18]:
def check_name_exists():
    phone = {"Dennis":89766, "Yann":78887, "Muriel":87765, "Tony":54443, "Julia":76665}
    print("Ready to check for names.")
    try:
        while True:
            name = yield
            if name in phone:
                print(f"The phone of {name} is {phone.get(name)}")
            else:
                print(f"Sorry, {name} not found")
    except ValueError:
        print("Exception ValueError raised")
        
coro = check_name_exists() 
next(coro)
while True:
    name=input("Enter a name: ")
    if len(name) <= 1:
        coro.throw(ValueError)
        break
    coro.send(name)

coro.close() # To stop the coroutine


Ready to check for names.
Enter a name: toto
Sorry, toto not found
Enter a name: marco
Sorry, marco not found
Enter a name: Tony
The phone of Tony is 54443
Enter a name: 
Exception ValueError raised


StopIteration: 

## yield from

When using a generator or a coroutine, you are not limited to only a single `yield`. You can, in fact, get other iterables, generators, or coroutines involved using **`yield from`**.

For example, let's say we want to implement a generator to generate Fibonacci sequence with limits, and decide to hardcode the first five elements of the sequence to get things started, we could write:

In [25]:
def fibonacci():
    starter = [1, 1, 2, 3, 5]
    yield from starter

    n1 = starter[-2]
    n2 = starter[-1]

    while True:
        yield (n := n1 + n2)
        n1, n2 = n2, n
        
for i in fibonacci():
    print(i, end=" ")
    if i > 100:
         break 

1 1 2 3 5 8 13 21 34 55 89 144 

In the above code, **:=** is the new "**walrus operator**" (new in Python 3.8), which assigns AND returns a value.
You can, also write the code this way:

`while True:
    n = n1 + n2
    yield n
    n1, n2 = n2, n`